In [1]:
import pandas as pd
import re
pd.options.display.max_columns = 30  #Can't have too many columns.
import glob
from datetime import datetime
import dateutil.parser
import nltk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']= (17,5)
from collections import Counter
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
import langid
from tqdm import tqdm
import os
import locale 

In [2]:
#Checking if all the speeches are loaded or not
folder = nltk.data.find('A:/Data Analysis Jupyter/Campaign-Analytics/Speeches/narendramodi/2014/')
path_link = 'A:/Data Analysis Jupyter/Campaign-Analytics/Speeches/narendramodi/2014/*'
paths = glob.glob(path_link)
len(paths)

38

In [3]:
nm_speeches = [ ]  #Empty dataframe to store all the content of speeches
for path in paths:
    corpusReader = nltk.corpus.PlaintextCorpusReader(folder, '.*\.txt')
    # opening the files and converting it to a dictionary
    with open(path, encoding="utf-8") as speech_file:
        filename = path.split('/')[-1]
        date = re.search(r"(_\d\d)_(\w+)_(\d\d\d\d)", filename) 
        if date:
            date = datetime.strptime(date.group(0), '_%d_%b_%Y')
        else:
            date = np.nan
        speech = {
            'Title' : path.split('/')[-1],
            'Speech' : speech_file.read(),
            'Date' : date
        }   
    nm_speeches.append(speech)
nm_df = pd.DataFrame(nm_speeches)
nm_df.head(10)

,Date,Speech,Title
0,2014-10-23,"प्‍यारे जवानों,आप सबको बड़ा सरप्राइज हुआ होगा ...",2014\11_23_Oct_2014_Text_of_PMs_address_to_Off...
1,2014-10-25,à¤à¤ªà¤¸à¥à¤¥à¤¿âà¤¤ à¤¸à¤­à¥ à¤µà¤°à¤¿à¤...,2014\12_25_Oct_2014_Text_of_PMs_address_at_the...
2,2014-10-31,à¤®à¥à¤°à¥ à¤¸à¤¾à¤¥ à¤à¤ª à¤²à¥à¤ à¤¬à¥...,2014\13_31_Oct_2014_Text_of_PMs_address_at_Run...
3,2014-10-31,Following is the English rendering of Prime Mi...,2014\14_31_Oct_2014_English_rendering_of_text_...
4,2014-11-02,"My dear fellow countrymen, I am with you again...",2014\15_02_Nov_2014_English_rendering_of_text_...
5,2014-11-02,à¤®à¥à¤°à¥ à¤ªà¥à¤¯à¤¾à¤°à¥ à¤¦à¥à¤¶à¤µà¤...,2014\16_02_Nov_2014_Text_of_PMs_Mann_ki_Baat_o...
6,2014-11-07,"मंच पर विराजमान राज्‍यपाल, श्रीमान राम नाइक जी...",2014\17_07_Nov_2014_Text_of_PMs_speech_at_Lalp...
7,2014-11-07,"उत्‍तर प्रदेश सरकार के मंत्री, श्रीमान अहमद हस...",2014\18_07_Nov_2014_Text_of_PMs_speech_at_Saan...
8,2014-11-08,आज यह घाट की सफाई का मैंने काम प्रारंभ किया है...,2014\19_08_Nov_2014_Text_of_PMs_message_after_...
9,2014-10-02,à¤­à¤¾à¤°à¤¤ à¤®à¤¾à¤¤à¤¾ à¤à¥ à¤à¤¯!à¤®à¤¹...,2014\1_02_Oct_2014_Text_of_PMs_address_during_...


In [4]:
nm_df['Year'],nm_df['Month'],nm_df['Day'] = nm_df.Date.dt.year, nm_df.Date.dt.month, nm_df.Date.dt.day

In [5]:
nm_df.head()

,Date,Speech,Title,Year,Month,Day
0,2014-10-23,"प्‍यारे जवानों,आप सबको बड़ा सरप्राइज हुआ होगा ...",2014\11_23_Oct_2014_Text_of_PMs_address_to_Off...,2014,10,23
1,2014-10-25,à¤à¤ªà¤¸à¥à¤¥à¤¿âà¤¤ à¤¸à¤­à¥ à¤µà¤°à¤¿à¤...,2014\12_25_Oct_2014_Text_of_PMs_address_at_the...,2014,10,25
2,2014-10-31,à¤®à¥à¤°à¥ à¤¸à¤¾à¤¥ à¤à¤ª à¤²à¥à¤ à¤¬à¥...,2014\13_31_Oct_2014_Text_of_PMs_address_at_Run...,2014,10,31
3,2014-10-31,Following is the English rendering of Prime Mi...,2014\14_31_Oct_2014_English_rendering_of_text_...,2014,10,31
4,2014-11-02,"My dear fellow countrymen, I am with you again...",2014\15_02_Nov_2014_English_rendering_of_text_...,2014,11,2


In [6]:
nm_df = nm_df[["Year","Month", "Day", "Title", "Speech", "Date"]]

In [7]:
nm_df.head()

,Year,Month,Day,Title,Speech,Date
0,2014,10,23,2014\11_23_Oct_2014_Text_of_PMs_address_to_Off...,"प्‍यारे जवानों,आप सबको बड़ा सरप्राइज हुआ होगा ...",2014-10-23
1,2014,10,25,2014\12_25_Oct_2014_Text_of_PMs_address_at_the...,à¤à¤ªà¤¸à¥à¤¥à¤¿âà¤¤ à¤¸à¤­à¥ à¤µà¤°à¤¿à¤...,2014-10-25
2,2014,10,31,2014\13_31_Oct_2014_Text_of_PMs_address_at_Run...,à¤®à¥à¤°à¥ à¤¸à¤¾à¤¥ à¤à¤ª à¤²à¥à¤ à¤¬à¥...,2014-10-31
3,2014,10,31,2014\14_31_Oct_2014_English_rendering_of_text_...,Following is the English rendering of Prime Mi...,2014-10-31
4,2014,11,2,2014\15_02_Nov_2014_English_rendering_of_text_...,"My dear fellow countrymen, I am with you again...",2014-11-02


In [8]:
nm_df.to_pickle('speeches_df_2014.pkl')